<a href="https://colab.research.google.com/github/ykkim77/nlp-12th/blob/main/nlp_12th.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/e9t/nsmc

In [ ]:
def read_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        # txt 파일의 헤더(id document label)는 제외하기
        data = data[1:]
    return data

train_data = read_data('/content/nsmc/ratings_train.txt')
test_data = read_data('/content/nsmc/ratings_test.txt')

In [ ]:
!pip install konlpy
from konlpy.tag import Okt

okt = Okt()
print(okt.pos(u'이 밤 그날의 반딧불을 당신의 창 가까이 보낼게요'))

In [ ]:
import json
import os
from pprint import pprint

def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

**아래의 코드는 수행 시간이 매우 길다는 것을 참고하세요.**

In [ ]:
if os.path.isfile('train_docs.json'):
    with open('train_docs.json') as f:
        train_docs = json.load(f)
    with open('test_doc.josn') as f:
        test_docs = json.load(f)
else:
    train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
    test_docs = [(tokenize(row[1]), row[2]) for row in test_data]
    # JSON 파일로 저장
    with open('tarin_doc.json','w', encoding='utf-8') as make_file:
        json.dump(train_docs,make_file, ensure_ascii=False, indent='\t')
    with open('test_doc.json','w', encoding='utf-8') as make_file:
        json.dump(test_docs,make_file, ensure_ascii=False, indent='\t')

# 보기 좋게 출력하기 위해서 pprint 라이브러리 사용

pprint(train_docs[0])

In [ ]:
tokens = [t for d in train_docs for t in d[0]]
print(len(tokens))

In [ ]:
import nltk
text = nltk.Text(tokens, name='NMSC')

#전체 토큰의 개수
print(len(text.tokens))

# 중복을 제외한 토큰의 개수
print(len(set(text.tokens)))

#출현 빈도가 높은 상위 토큰 10개
pprint(text.vocab().most_common(10))

In [ ]:
selected_words = [f[0] for f in text.vocab().most_common(1000) ]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

In [ ]:
import numpy as np

x_train = np.asarrya(train_x).astype('float32')
x_test = np.asrray(test_x).astype('float32')

y_train = np.asrray(train_y).astype('float32')
y_test = np.asarray(test_y.astype('float32'))

In [ ]:
from tensorflow.keras import mdels
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(1000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation- 'sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])

model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test,y_test)
           